# Solution: Practice QBUS3840 Test using the Heater dataset

You will anayze a classic dataset, from Kenneth Train, about Heating system preferences.
Households face the decision of which heating system to install, for example, Should we install a gas central heating system or for a heat pump instead? (among others).

Several variables that affect the decision are measured, such as the installation costs, operation costs for one year and characteristics such as the number of rooms in the house or the income of the household.
The dataset was gathered in California



## Description of the dataset

Each row represents a different household, os the household is the 'decision-maker' in this scenario. Household are 'independent' of each other.

The variables in the dataset are:

**idcase:** The identifier of each individual, decision-maker.

**depvar**: A categorical variable indicating the choice of heating system. It is encoded in text *(we will turn it into numbers for biogeme in the preprocessing step)*. We have 5 alternatives.

 * 'gc': Gas central
 * 'gr': Gas room
 * 'ec': Electric central
 * 'er': Electric room
 * 'hp': Heat pump

**Installation costs variables**:  The cost of installing each system, the variable names are encoded such as `ic_xx`, with xx being the name of the alternative, as in the depvar variable. For example the column `ic_gc` means installation costs for the gas central alternative. `ic_er` would be installation cost for the electric room alternative.

**Operation costs**: Operation costs of each heating system, for a year. The variable names are encoded in a similar fashion to installation cost. So the column`oc_gr` would mean operation cost for the gas room alternative.

**rooms**: The number of rooms in the house, a numeric variable.

**agehed**: Age of the decision maker, considered as the 'household head'.

**income**: Yearly income of the household, in dollars.

**region**: A categorical variable encoding the location of the household within California. Four levels encoded with text (will be turned into numbers for processing in biogeme).
 * 'ncostl': Norther coastal region
 * 'scostl': Souther coastal region
 * 'mountn': Mountain region
 * 'valley': Valley region

---
---

# Preparing the environment


In [ ]:
!pip install biogeme

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Load the packages, feel free to change the names.

In [ ]:
import pandas  as pd
import numpy as np
import matplotlib.pyplot as plt

import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
import biogeme.expressions as exp
import biogeme.tools as tools

# Load the dataset

In [ ]:

heater_pd = pd.read_csv('https://github.com/pmontman/tmp_choicemodels/raw/main/data/heating.csv')


A simple look at the dataset.

In [ ]:
heater_pd.head(5)

,idcase,depvar,ic_gc,ic_gr,ic_ec,ic_er,ic_hp,oc_gc,oc_gr,oc_ec,oc_er,oc_hp,income,agehed,rooms,region
0,1,gc,866.00,962.64,859.90,995.76,1135.50,199.69,151.72,553.34,505.60,237.88,7,25,6,ncostl
1,2,gc,727.93,758.89,796.82,894.69,968.90,168.66,168.66,520.24,486.49,199.19,5,60,5,scostl
2,3,gc,599.48,783.05,719.86,900.11,1048.30,165.58,137.80,439.06,404.74,171.47,4,65,2,ncostl
3,4,er,835.17,793.06,761.25,831.04,1048.70,180.88,147.14,483.00,425.22,222.95,2,50,4,scostl
4,5,er,755.59,846.29,858.86,985.64,883.05,174.91,138.90,404.41,389.52,178.49,2,25,6,valley


Data cleaning (not needed in a exam): The variable depvar uses strings for the variable, we need to use integers (starting in 1) for biogeme. So we re-encode the `depvar` variable as integer using the pandas `factorize` function.

**Be careful with the encoding! according to `factorize`, in this dataset, the corresponding numbers will be:**
 1. gas central
 2. electricity room
 3. gas room
 4. heat pump
 5. electricty central

---



In [ ]:
depvar_factor = pd.factorize(heater_pd['depvar'])

heater_pd['depvar'] = depvar_factor[0] + 1
depvar_factor[1]

Index(['gc', 'er', 'gr', 'hp', 'ec'], dtype='object')

The `region` variable, we will encoded it as numbers via binary encoding. We do this with `get_dummies` function from pandas. We can do the efficient binary encoding, considering one of the levels of region as the baseline (saving one variable), or the explicit encoding, creating one variable per level.
Let's go for explicit encoding, easier interpretation.

We will also print a snapshot of the resulting dataset, already clean and ready for analysis.

In [ ]:
heater_pd = pd.get_dummies(heater_pd, 'region')

heater_pd.head(5)

,idcase,depvar,ic_gc,ic_gr,ic_ec,ic_er,ic_hp,oc_gc,oc_gr,oc_ec,oc_er,oc_hp,income,agehed,rooms,region_mountn,region_ncostl,region_scostl,region_valley
0,1,1,866.00,962.64,859.90,995.76,1135.50,199.69,151.72,553.34,505.60,237.88,7,25,6,0,1,0,0
1,2,1,727.93,758.89,796.82,894.69,968.90,168.66,168.66,520.24,486.49,199.19,5,60,5,0,0,1,0
2,3,1,599.48,783.05,719.86,900.11,1048.30,165.58,137.80,439.06,404.74,171.47,4,65,2,0,1,0,0
3,4,2,835.17,793.06,761.25,831.04,1048.70,180.88,147.14,483.00,425.22,222.95,2,50,4,0,0,1,0
4,5,2,755.59,846.29,858.86,985.64,883.05,174.91,138.90,404.41,389.52,178.49,2,25,6,0,0,0,1


---
---

# 1) Adjust a model with alternative specific constants and shared parameters for installation cost and operation costs. Select one of the alternatives as the reference (pick the one that you prefer). Comment on the results: Signs of the variables and alternative specific constants.

In [ ]:
bgm_heater = db.Database('heater', heater_pd)
globals().update(bgm_heater.variables)


In [ ]:
ASC_GC = exp.Beta ( 'ASC_GC' ,0, None , None ,0)
ASC_ER = exp.Beta ( 'ASC_ER' ,0, None , None ,0)
ASC_GR = exp.Beta ( 'ASC_GR' ,0, None , None ,0)
ASC_HP = exp.Beta ( 'ASC_HP' ,0, None , None ,1)
ASC_EC = exp.Beta ( 'ASC_EC' ,0, None , None ,0)
B_INSTCOST = exp.Beta ( 'B_INSTCOST' ,0, None , None ,0)
B_OPERCOST = exp.Beta ( 'B_OPERCOST' ,0, None , None ,0)

In [ ]:
V1 = ASC_GC + B_INSTCOST*ic_gc + B_OPERCOST*oc_gc
V2 = ASC_ER + B_INSTCOST*ic_er + B_OPERCOST*oc_er
V3 = ASC_GR + B_INSTCOST*ic_gr + B_OPERCOST*oc_gr
V4 = ASC_HP + B_INSTCOST*ic_hp + B_OPERCOST*oc_hp
V5 = ASC_EC + B_INSTCOST*ic_ec + B_OPERCOST*oc_ec

In [ ]:
V = {1: V1 ,
2: V2 ,
3: V3,
4: V4,
5: V5 }

av = {1: 1,
2: 1,
3: 1,
4: 1,
5: 1 }

In [ ]:
logprob_base = models.loglogit (V , av , depvar )

In [ ]:
bgm_basemodel = bio.BIOGEME ( bgm_heater, logprob_base )

In [ ]:
results_base = bgm_basemodel.estimate()

[03:33:23] < Warning >   You have not defined a name for the model. The output files are named from the model name. The default is [biogemeModelDefaultName]


In [ ]:
results_base.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_EC,1.658845,0.439866,3.771247,1.624340e-04
ASC_ER,1.853436,0.349149,5.308442,1.105663e-07
ASC_GC,1.710979,0.221413,7.727546,1.088019e-14
ASC_GR,0.308262,0.206334,1.493995,1.351769e-01
B_INSTCOST,-0.001533,0.000607,-2.526874,1.150828e-02
B_OPERCOST,-0.006996,0.001468,-4.764474,1.893469e-06


---
---

# 2) Calculate the willingness to pay for reducing operating cost.
* *In this case, we have two 'price' variables (installation and operation): Operating cost is the variable we want to understand, installation cost is the price variable in the WTP formula.*

Apply the definition, for a purely linear specification, the ratio of the coefficients.

The number seems reasonable, people are willing to pay 4.5 dollars in installation to reduce the annual operation cost in one dollar.


In [ ]:
results_base.getBetaValues()['B_OPERCOST']  / results_base.getBetaValues()['B_INSTCOST']

4.563382574989451

---
---

#3) Do big houses with many rooms (5,6,7) have different preferences than the rest (4 and less rooms)?
* *Might or might not need to fit another model.*

Several ways of answering these:
* just compare the two groups, computing the proportions of each alternative in each group
* Fit a model to each group, compare the models
* Fit a model with the indicator variable 'more than 4 rooms' as constant
* Fit a model with interactions with an indicator variable 'more than 4 rooms', compare to the reference

The idea here is to highlight the difference with question 5. Question 5 will ask specifically for the **interaction** of the 'region_valley' indicator variable with installation cost and operation cost, to check wether the coefficients for installation cost and operation cost are different for the people in the valley, compared to the others. This Question 3 is simples, just compare if the 'overall preferences' are different, not that the effects of a particular attributes on the utility are different.
Question 3 ask if the preferences between groups are different, Question 5 is the individuals are different.

In [ ]:
pd.crosstab(heater_pd[heater_pd['rooms'] >4]['depvar'], 'depvar', normalize='all')

col_0,depvar
depvar,
1,0.627907
2,0.104651
3,0.130233
4,0.058140
5,0.079070


In [ ]:
pd.crosstab(heater_pd[heater_pd['rooms'] <=4]['depvar'], 'depvar', normalize='all')

col_0,depvar
depvar,
1,0.644681
2,0.082979
3,0.155319
4,0.053191
5,0.063830


---
---

# 4) Create a more complex model, that includes at least one *interaction* variable between an attribute and a characteristic (product of two variables). Comment on the interpretation of the model. Comment of the per-alernative Willingess To Pay for operating cost, and how they compare to the answer in Question 2.
# Is the model a better fit than the one created in in Question 1?


We simplify a bit, Question 4 is asking us to fit an interaction term (product of two variables), and Question 5 can be solved via an interaction term, with 'region_valley' variable, so we will specify that interaction here to save time and not fit separate models in Questions 4 and 5, we will just fit one model.

In [ ]:
B_OPER_VALLEY_GC = exp.Beta ( 'B_OPER_VALLEY_GC' ,0, None , None ,0)
B_OPER_VALLEY_ER = exp.Beta ( 'B_OPER_VALLEY_ER' ,0, None , None ,0)
B_OPER_VALLEY_GR = exp.Beta ( 'B_OPER_VALLEY_GR' ,0, None , None ,0)
B_OPER_VALLEY_HP = exp.Beta ( 'B_OPER_VALLEY_HP' ,0, None , None ,0)
B_OPER_VALLEY_EC = exp.Beta ( 'B_OPER_VALLEY_EC' ,0, None , None ,0)

B_INST_VALLEY_GC = exp.Beta ( 'B_INST_VALLEY_GC' ,0, None , None ,0)
B_INST_VALLEY_ER = exp.Beta ( 'B_INST_VALLEY_ER' ,0, None , None ,0)
B_INST_VALLEY_GR = exp.Beta ( 'B_INST_VALLEY_GR' ,0, None , None ,0)
B_INST_VALLEY_HP = exp.Beta ( 'B_INST_VALLEY_HP' ,0, None , None ,0)
B_INST_VALLEY_EC = exp.Beta ( 'B_INST_VALLEY_EC' ,0, None , None ,0)

In [ ]:
V1_inter = ASC_GC + B_INSTCOST*ic_gc + B_OPERCOST*oc_gc +  B_INST_VALLEY_GC*ic_gc*region_valley + B_OPER_VALLEY_GC*oc_gc*region_valley
V2_inter = ASC_ER + B_INSTCOST*ic_er + B_OPERCOST*oc_er +  B_INST_VALLEY_ER*ic_er*region_valley + B_OPER_VALLEY_ER*oc_er*region_valley
V3_inter = ASC_GR + B_INSTCOST*ic_gr + B_OPERCOST*oc_gr +  B_INST_VALLEY_GR*ic_gr*region_valley + B_OPER_VALLEY_GR*oc_gr*region_valley
V4_inter = ASC_HP + B_INSTCOST*ic_hp + B_OPERCOST*oc_hp +  B_INST_VALLEY_HP*ic_hp*region_valley + B_OPER_VALLEY_HP*oc_hp*region_valley
V5_inter = ASC_EC + B_INSTCOST*ic_ec + B_OPERCOST*oc_ec +  B_INST_VALLEY_EC*ic_ec*region_valley + B_OPER_VALLEY_EC*oc_ec*region_valley

In [ ]:
V_inter = {1: V1_inter ,
2: V2_inter ,
3: V3_inter,
4: V4_inter,
5: V5_inter }

In [ ]:
logprob_inter = models.loglogit (V_inter , av , depvar )
bgm_intermodel = bio.BIOGEME ( bgm_heater, logprob_inter )
results_inter = bgm_intermodel.estimate()
results_inter.getEstimatedParameters()

[03:33:23] < Warning >   You have not defined a name for the model. The output files are named from the model name. The default is [biogemeModelDefaultName]


,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_EC,1.787048,0.473729,3.772299,1.617505e-04
ASC_ER,1.954662,0.377192,5.182147,2.193464e-07
ASC_GC,1.737270,0.254128,6.836192,8.132606e-12
ASC_GR,0.309719,0.236475,1.309730,1.902872e-01
B_INSTCOST,-0.001622,0.000696,-2.328873,1.986582e-02
B_INST_VALLEY_EC,0.001619,0.004566,0.354644,7.228566e-01
B_INST_VALLEY_ER,-0.000489,0.002341,-0.208966,8.344749e-01
B_INST_VALLEY_GC,-0.000433,0.002128,-0.203538,8.387148e-01
B_INST_VALLEY_GR,0.002052,0.002039,1.006384,3.142311e-01
B_INST_VALLEY_HP,-0.001328,0.002788,-0.476395,6.337930e-01


### WTP:
 Because we have an interaction term with the cost varaibles, the model is not linear with respect to costs. If we apply the definition of WTP, we would get two WTP per alternative, one for the 'valley region' individuals and one for the 'non-valley region' individuals. The model simplifies for the nonvalley because their value for the 'region_valley' variable is 0, so the WTP for them is just the ratio.

 If you chose a non dummy variable for the interaction, such as income, the you would get a different WTP per individual (almost) so the answer should be a plot or a function, or a list of values.

For the dummy interaction with valley, the WTP for the non-valley individuals is just the usual one.

In [ ]:
results_inter.getBetaValues()['B_OPERCOST'] / results_inter.getBetaValues()['B_INSTCOST']

4.556015796724559

For the valley individuals, their 'different' Operation cost and installation cost coefficients can be calculated by adding the interaction terms to the 'common' coefficients (the ones we used for the non-valley).
The resulting WTP look really strange, a very large one!
Might be due to a lack of sample size? Maybe we should specify a better model, such different operation cost and installation cost coefficient per alternative, not only the interaction term?
Anyways this part would be OK even though the numbers look weird.

In [ ]:
(results_inter.getBetaValues()['B_OPERCOST'] + results_inter.getBetaValues()['B_OPER_VALLEY_GC'] ) / (results_inter.getBetaValues()['B_INSTCOST'] + results_inter.getBetaValues()['B_INST_VALLEY_GC'] )

-0.2200865837599923

In [ ]:
(results_inter.getBetaValues()['B_OPERCOST'] + results_inter.getBetaValues()['B_OPER_VALLEY_ER'] ) / (results_inter.getBetaValues()['B_INSTCOST'] + results_inter.getBetaValues()['B_INST_VALLEY_ER'] )

1.6208774481984616

In [ ]:
(results_inter.getBetaValues()['B_OPERCOST'] + results_inter.getBetaValues()['B_OPER_VALLEY_GR'] ) / (results_inter.getBetaValues()['B_INSTCOST'] + results_inter.getBetaValues()['B_INST_VALLEY_GR'] )

-28.329684964163683

In [ ]:
(results_inter.getBetaValues()['B_OPERCOST'] + results_inter.getBetaValues()['B_OPER_VALLEY_HP'] ) / (results_inter.getBetaValues()['B_INSTCOST'] + results_inter.getBetaValues()['B_INST_VALLEY_HP'] )

-1.6872823205086105

In [ ]:
(results_inter.getBetaValues()['B_OPERCOST'] + results_inter.getBetaValues()['B_OPER_VALLEY_EC'] ) / (results_inter.getBetaValues()['B_INSTCOST'] + results_inter.getBetaValues()['B_INST_VALLEY_EC'] )

3431.7841836980565

### Comparing to the base model
It is not significatively different to the base model, according to the likelihood ratio test at 5% significance.

In [ ]:
tools.likelihood_ratio_test( (results_inter.data.logLike, results_inter.data.nparam),
                            (results_base.data.logLike, results_base.data.nparam), 0.05)

LRTuple(message='H0 cannot be rejected at level 5.0%', statistic=3.5085370182514453, threshold=18.307038053275146)

---
---

# 5) Do the people of the 'valley' region have significatively **DIFFERENT** utility relationship for installation cost and operation cost, compared to the other regions? (Apologies the important word *different* was missing from the question!)
*You might need to fit one model (or two) to answer this question.*

Again several ways to solve it:
* One way is to estimate a model with interactions, interacting with the 'region_valley' variable, and then use a statistical test.
* Another way is to separate the datasets and create one dataset only for the valley people. Then use a statistical test to compare the model fit to the valley people and a model fit to the non-valley people**

We use the interaction approach, and reuse the specification for question 4, the specification for question 4 was chosen to answer this!**

So the solution for this one will be just, the same comparison as in Question 3, a likelihood ratio test comparing to the base model in Question 1

In [ ]:
tools.likelihood_ratio_test( (results_inter.data.logLike, results_inter.data.nparam),
                            (results_base.data.logLike, results_base.data.nparam), 0.05)

LRTuple(message='H0 cannot be rejected at level 5.0%', statistic=3.5085370182514453, threshold=18.307038053275146)

---
---

# 6) Due to a 'Special Operation', it is expected that the supply of gas will be completely cut, this is, the two alternatives that use gas, 'gas central' and 'gas room' will not be available. The households that *chose one of those alternatives* will have to move another heating system. Calculate the installation cost that will be incurred due to this change for the population. Use the model fitted in Question 4.

One way to emulate the removal of some alternatives is to set the costs to a very high value, assuming that the probabilities are going to be negatively impacted.
The second part is to focus only on the individuals that chose alternatives 'gas central' or 'gas room'.

In [ ]:
wprob_gc = models.logit(V_inter, av, 1)
wprob_er = models.logit(V_inter, av, 2)
wprob_gr = models.logit(V_inter, av, 3)
wprob_hp = models.logit(V_inter, av, 4)
wprob_ec = models.logit(V_inter, av, 5)


wtargets_to_simulate ={'Prob. gc':  wprob_gc ,
                      'Prob. er':  wprob_er ,
           'Prob. gr': wprob_gr,
           'Prob. hp': wprob_hp ,
           'Prob. ec': wprob_ec  }



Lets calculate the original choice probabilties

In [ ]:
w_preds = bio.BIOGEME( bgm_heater, wtargets_to_simulate).simulate(results_inter.getBetaValues())
w_preds.mean(0)

Prob. gc    0.636666
Prob. er    0.093333
Prob. gr    0.143334
Prob. hp    0.055557
Prob. ec    0.071110
dtype: float64

We will modify the dataset to remove gas alternatives, to see the difference

In [ ]:
heater_pd_nogas = heater_pd.copy()
heater_pd_nogas['ic_gc'] *= 1000
heater_pd_nogas['ic_gr'] *= 1000
heater_pd_nogas['oc_gc'] *= 1000
heater_pd_nogas['oc_gr'] *= 1000


w_preds_nogas = bio.BIOGEME( db.Database('heater_nogas', heater_pd_nogas), wtargets_to_simulate).simulate(results_inter.getBetaValues())
w_preds_nogas.mean(0)

Prob. gc    0.000000
Prob. er    0.418414
Prob. gr    0.000000
Prob. hp    0.262214
Prob. ec    0.319372
dtype: float64

Another way of 'calculating' the removal of alternatives is by applying the properties of the multinomial logit, we know that by the Independence of Irrelevant alternatives, the ratio of probabilities (odds) stays constant when removing 'irrelevant' alternatives. For example, we can check the two datasets, we see that the ratio of proportions for 'er' and 'hp' alternatives is the same, even when removing the gas alternatives.

In [ ]:
np.mean(w_preds['Prob. er'] / w_preds['Prob. hp'] )

1.7574884989949622

In [ ]:
np.mean(w_preds_nogas['Prob. er'] / w_preds_nogas['Prob. hp'] )

1.7574884989949624

So we could find out the new choice probabilities in an 'analytical' way, without simulation. The easy way is through simulation :).


---
We continue, we get the subset of people in the dataset that chose gas alternatives. These are the individuals that will have to reinstall their heating system.

In [ ]:
heater_pd_nogas_reinstall = heater_pd_nogas[ (heater_pd_nogas['depvar'] == 1) | (heater_pd_nogas['depvar'] == 3) ]

Check the choice probabilities (not required, just for clarity).

In [ ]:
w_preds_nogas_reinstall = bio.BIOGEME( db.Database('heater_remove', heater_pd_nogas_reinstall), wtargets_to_simulate).simulate(results_inter.getBetaValues())
w_preds_nogas_reinstall.mean(0)

Prob. gc    0.000000
Prob. er    0.416203
Prob. gr    0.000000
Prob. hp    0.265862
Prob. ec    0.317936
dtype: float64

Sum the installation cost times the choice probabilities to get an estimation of the total cost

In [ ]:
(w_preds_nogas_reinstall.to_numpy() * heater_pd_nogas_reinstall[ ['ic_gc', 'ic_er', 'ic_gr', 'ic_hp', 'ic_ec']].to_numpy()).sum(axis=None)

669053.3844121457